# Install

In [ ]:
!pip install gradio
!apt install fonts-noto-cjk

## Tesseract

In [ ]:
!apt install tesseract-ocr libtesseract-dev tesseract-ocr-jpn
!pip install pytesseract

## PaddleOCR

In [ ]:
!pip install paddlepaddle
!pip install paddleocr

## EasyOCR

In [ ]:
!pip install easyocr

# Demo

In [ ]:
import torch
gpu = True if torch.cuda.is_available() else False

In [ ]:
from PIL import ImageFont, ImageDraw, Image

def putText(img, text, point, size, color):
  font = ImageFont.truetype('/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc', size)
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text(point, text, fill=color, font=font)
  return np.array(img_pil)

In [ ]:
import cv2
import numpy as np

def getImage(img, results):
  fx = 600/img.shape[0]
  img_box = cv2.resize(img, None, None, fx, fx)
  img_text = np.ones((img.shape[0], img.shape[1], 3), np.uint8) * 255
  for left, top, right, bottom, text in  results:
    if type(text) == str:
      cv2.rectangle(img_box, (int(left*fx), int(top*fx)), (int(right*fx), int(bottom*fx)), (255, 0, 0), thickness=2)
      img_text = putText(img_text, text, (left, top), int((right-left)/len(text)), (255, 0, 0))

  return img_box, img_text

## Tesseract

In [ ]:
import time
from pytesseract import pytesseract

def ocr_tesseract(img):
  time_start = time.perf_counter()
  data = pytesseract.image_to_data(img, lang="jpn", config='--psm 12', output_type=pytesseract.Output.DATAFRAME)
  time_end = time.perf_counter()

  results = []
  for left, top, width, height, text in  zip(data['left'], data['top'], data['width'], data['height'], data['text']):
    if type(text) == str:
      results.append((left, top, left+width, top+height, text))

  img_box, img_text = getImage(img, results)
  return time_end-time_start, img_box, img_text

## PaddleOCR

In [ ]:
import time
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_gpu=gpu, lang = "japan")

def ocr_paddleocr(img):
  time_start = time.perf_counter()
  ocr = PaddleOCR(use_gpu=gpu, lang = "japan", det_limit_side_len=img.shape[0], max_text_length = 30,)
  data = ocr.ocr(img = img, det=True, rec=True, cls=False)
  time_end = time.perf_counter()

  results = []
  for result in data[0]:
    if result:
      results.append((int(result[0][0][0]), int(result[0][0][1]), int(result[0][2][0]), int(result[0][2][1]), result[1][0]))

  img_box, img_text = getImage(img, results)
  return time_end-time_start, img_box, img_text

## EasyOCR

In [ ]:
import time
import easyocr
reader = easyocr.Reader(['ja','en'], gpu=gpu)

def ocr_easyocr(img):
  time_start = time.perf_counter()
  data = reader.readtext(img)
  time_end = time.perf_counter()

  results = []
  for result in data:
    if result:
      results.append((int(result[0][0][0]), int(result[0][0][1]), int(result[0][2][0]), int(result[0][2][1]), result[1]))

  img_box, img_text = getImage(img, results)
  return time_end-time_start, img_box, img_text

## Gradio

In [ ]:
def ocr(img):
  # Tesseract
  txt_tes_1, img_tes_1, img_tes_2 = ocr_tesseract(img)

  # PaddleOCR
  txt_pad_1, img_pad_1, img_pad_2 = ocr_paddleocr(img)

  # EasyOCR
  txt_esy_1, img_esy_1, img_esy_2 = ocr_easyocr(img)

  return txt_tes_1, img_tes_1, img_tes_2, txt_pad_1, img_pad_1, img_pad_2, txt_esy_1, img_esy_1, img_esy_2

In [ ]:
import gradio as gr

css = ".img_out {height: 600px !important} "
with gr.Blocks(css=css) as demo:
  with gr.Row():
    input_img = gr.Image()
  with gr.Row():
    btn = gr.Button("Exec!")
  with gr.Row():
    output_tes_1 = gr.Textbox(label="Tesseract")
  with gr.Row():
    output_tes_2 = gr.Image(elem_classes='img_out', label="Tesseract")
    output_tes_3 = gr.Image(elem_classes='img_out', label="Tesseract")
  with gr.Row():
    output_pad_1 = gr.Textbox(label="PaddleOCR")
  with gr.Row():
    output_pad_2 = gr.Image(elem_classes='img_out', label="PaddleOCR")
    output_pad_3 = gr.Image(elem_classes='img_out', label="PaddleOCR")
  with gr.Row():
    output_esy_1 = gr.Textbox(label="EasyOCR")
  with gr.Row():
    output_esy_2 = gr.Image(elem_classes='img_out', label="EasyOCR")
    output_esy_3 = gr.Image(elem_classes='img_out', label="EasyOCR")
  btn.click(fn=ocr, inputs=[input_img], outputs=[
      output_tes_1, output_tes_2, output_tes_3,
      output_pad_1, output_pad_2, output_pad_3,
      output_esy_1, output_esy_2, output_esy_3,
      ])

demo.launch(debug=False)